# Format and merge spectra energies together

We are using the pyteomics library to read and write the MGF format (https://pyteomics.readthedocs.io/en/latest/). 

TODO: maybe use the matchms library instead
EDIT: Both are OK. But pyteomics is easier to use if we just want to clean or modify rapidly the mgf. Where matchms is more for dealing with the intensities, like normalization, modification, etc... 

In [58]:
from pyteomics import mgf, auxiliary
import numpy as np
from collections import defaultdict
from tqdm import tqdm

In [ ]:
# Load the mgf
infile = "../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.mgf"

Structure view of the current MGF

In [ ]:
with mgf.read(infile) as reader: 
    auxiliary.print_tree(next(reader))

In [ ]:
# Go through the spectra
with mgf.read(infile) as reader: 
    for i, spectrum in enumerate(reader):
        if i > 0: break
        print(i, spectrum)
        #spectrum["params"]["title"] = spectrum["params"]["title"]
        print(spectrum["m/z array"], spectrum['intensity array'])
        print(dict(zip(spectrum["m/z array"], spectrum['intensity array'])))
        

### Step for merging the energies together

In [ ]:
from collections import defaultdict
from tqdm import tqdm

modified_spectra = defaultdict(list)
all_mz_intensity_dict = defaultdict(dict)

# input MGF 
infile = "../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.mgf"

with mgf.read(infile, convert_arrays = 1, read_charges = False) as infile:
    for spectrum in tqdm(infile):
        # careful when splitting, inchis have ";" in them... 
        try:
            id, energy, type, cfmid_version, *inchi = spectrum["params"]["title"].split(";")
        except ValueError:
            print(spectrum)
        spectrum["params"]["title"] = id
        spectrum["params"]["cfmid"] = cfmid_version
        spectrum["params"]["InChI"] = ';'.join(inchi)[:-1]
        spectrum["params"]["name"] = id
        mz_array = spectrum["m/z array"]
        intensity_array = spectrum['intensity array']
        current_mz_intensity_dict = dict(zip(mz_array, intensity_array))
    
        if id not in modified_spectra.keys():
            #print(modified_spectra.keys())
            #print("Added new spectra for", id, "energy", energy)
            #print(current_mz_intensity_dict)
            modified_spectra[id] = spectrum
            all_mz_intensity_dict[id] = current_mz_intensity_dict
        else:
            #print("Merged intensities for spectra", id, "energy", energy)
            all_mz_intensity_dict[id] = dict(sorted((*all_mz_intensity_dict[id].items(), *current_mz_intensity_dict.items())))
            modified_spectra[id]["m/z array"] = list(all_mz_intensity_dict[id].keys())
            modified_spectra[id]["intensity array"] = list(all_mz_intensity_dict[id].values())
            #print(*all_mz_intensity_dict[id].items())
            #print(all_mz_intensity_dict[id])

print(len(modified_spectra.keys()))

In [ ]:
outfile = "../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.energies_merged_name.mgf"
!rm -f $outfile

# mgf.write always appends to the file, so we need to remove it if we run it again
mgf.write(spectra=modified_spectra.values(), output=outfile)

In [ ]:
# Check the output file
!head $outfile

!grep -c "TITLE" $outfile

#### Convert spectra to GNPS format and create the batchupload tsv file

In order to send our data to GNPS, we have to reformat our MGF like this:

- https://gnps.ucsd.edu/ProteoSAFe/result.jsp?task=1ad7bc366aef45ce81d2dfcca0a9a5e7&view=download_clustered_spectra

And add a tsv file as they show on this link:

- https://ccms-ucsd.github.io/GNPSDocumentation/batchupload/


In [ ]:
import sqlite3
import pandas as pd

DB_PATH = "../../Database/compounds.sqlite"

con = sqlite3.connect(DB_PATH)
cur = con.cursor()

compounds_info = pd.read_sql_query('SELECT ID, SMILES FROM compounds', con)

# We need to add the smile code, so we extract it from our database 
# And use a dict consisting only of {ID: SMILES} to retrieve them fast based on ID
series = compounds_info["SMILES"]
series.index = compounds_info["ID"]
compounds_dict = series.to_dict()

In [99]:
from dataclasses import dataclass

@dataclass
class Entry:
    filename: str
    compound_name: str
    moleculemass: float
    extractscan: int
    smiles: str
    inchi: str
    seq: str = "*..*"
    instrument: str = "Hybrid F"
    ionsource: str = "LC-ESI"
    inchiaux: str = "N/A"
    charge: int = 1
    ionmode: str = "Positive"
    pubmed: str = "N/A"
    acquisition: str = "Other"
    exactmass: float = 0
    datacollector: str = "D. Elser, D. Pflieger"
    adduct: str = "M+H"
    interest: str = "N/A"
    libquality: int = 3
    genus: str = "N/A"
    species: str = "N/A"
    strain: str = "N/A"
    casnumber: str = "N/A"
    pi: str = "E. Gaquerel"

    def fields(self):
        fields = [self.filename, self.seq, self.compound_name, self.moleculemass, self.instrument, self.ionsource, self.extractscan, self.smiles, 
            self.inchi, self.inchiaux, self.charge, self.ionmode, self.pubmed, self.acquisition, self.exactmass, self.datacollector, 
             self.adduct, self.interest, self.libquality, self.genus, self.species, self.strain, self.casnumber, self.pi]
        return fields  
        

In [100]:
import csv

infile = "../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.energies_merged_name.mgf"
outfile = "../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.energies_merged_name.GNPS_like.mgf"
outtsv = '../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.energies_merged_name.GNPS_like.tsv'

modified_spectra = defaultdict(list)

with mgf.read(infile, convert_arrays = 1, read_charges = False) as infile:
    with open(outtsv, "w") as out:
        # Required fields
        header = ["FILENAME", "SEQ", "COMPOUND_NAME", "MOLECULEMASS", "INSTRUMENT", "IONSOURCE", "EXTRACTSCAN", 
                  "SMILES", "INCHI", "INCHIAUX", "CHARGE", "IONMODE", "PUBMED", "ACQUISITION", "EXACTMASS", "DATACOLLECTOR", 
                  "ADDUCT", "INTEREST", "LIBQUALITY", "GENUS", "SPECIES", "STRAIN", "CASNUMBER", "PI"]
        # CSV writer
        writer = csv.writer(out, delimiter='\t')
        writer.writerow(header)
        
        # Loop through the spectra
        for index, spectrum in enumerate(tqdm(infile), start = 1):
            id = spectrum["params"]["title"]
            smile = compounds_dict[id]
            
            Compound = Entry(filename = "TOTAL_COMPOUNDS_DB.energies_merged_name.GNPS_like.mgf", 
                             compound_name = id, 
                             moleculemass = spectrum["params"]["pepmass"][0], 
                             extractscan = index, 
                             smiles = smile, 
                             inchi = spectrum["params"]["inchi"])
                             
            writer.writerow(Compound.fields())

            spectrum["params"]["title"] = f"Scan Number: {index}"
            spectrum["params"]["filename"] = Compound.filename
            spectrum["params"]["scans"] = index
            del spectrum["params"]["inchi"]

            modified_spectra[id] = spectrum

print("Saving MGF now...")
mgf.write(spectra=modified_spectra.values(), output=outfile)
print("MGF was succesfully saved!")

100%|██████████| 1066512/1066512 [03:30<00:00, 5059.05it/s]


Saving MGF now...
MGF was succesfully saved!


### Using matchms 

In [87]:
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf

infile = "../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.energies_merged_name.mgf"
outfile = "../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.energies_merged_name.GNPS_like.matchms.mgf"
outcsv = '../../Ressources/CFMID/MGF/TOTAL_COMPOUNDS_DB.energies_merged_name.GNPS_like.matchms.csv'

modified_spectra = []
spectrums = load_from_mgf(infile)

with open(outcsv, "w") as out:
    header = ["FILENAME", "SEQ", "COMPOUND_NAME", "MOLECULEMASS", "INSTRUMENT", "IONSOURCE", "EXTRACTSCAN", 
              "SMILES", "INCHI", "INCHIAUX", "CHARGE", "IONMODE", "PUBMED", "ACQUISITION", "EXACTMASS", "DATACOLLECTOR", 
              "ADDUCT", "INTEREST", "LIBQUALITY", "GENUS", "SPECIES", "STRAIN", "CASNUMBER", "PI"]
    writer = csv.writer(out)
    writer.writerow(header)
    
    for index, spectrum in enumerate(tqdm(spectrums), start = 1):
        id = spectrum.get("title")
        smile = compounds_dict[id]

        Compound = Entry(filename = "TOTAL_COMPOUNDS_DB.energies_merged_name.GNPS_like.mgf", 
                         compound_name = id, 
                         moleculemass = spectrum.get("pepmass")[0], 
                         extractscan = index, 
                         smiles = smile, 
                         inchi = spectrum.get("inchi"))

        writer.writerow(Compound.fields())

        spectrum.set("title", f"Scan Number: {index}")
        spectrum.set("filename",  Compound.filename)
        spectrum.set("scans", index)
        #spectrum.set("smiles", smiles)
                         
        modified_spectra.append(spectrum)

save_as_mgf(spectrums = modified_spectra, filename = outfile)                     


1066512it [04:34, 3880.83it/s]
